In [1]:
import autograd.numpy as np

# this is needed to compensate for %matplotl+ib notebook's tendancy to blow up images when plotted inline
from matplotlib import rcParams
rcParams['figure.autolayout'] = True
%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import random
import matplotlib.pyplot as plt
from autograd import grad
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import Image
from scipy.signal import find_peaks
from sklearn.preprocessing import MinMaxScaler

from numpy.fft import *
import matplotlib.style as style 
style.use('ggplot')

# import function flattening module from autograd
from autograd.misc.flatten import flatten_func

import math

In [2]:
data = np.loadtxt('../data/test.csv',delimiter = ' ')[0]

In [3]:
#function to filter out frequencies in signal that are above the threshold
def filter_signal(signal, threshold=5e3):
    fourier = rfft(signal)
    frequencies = rfftfreq(signal.size, d=20e-3/signal.size)
    fourier[frequencies > threshold] = 0
    return irfft(fourier)

In [4]:
#Plot of a clean window of data
clean_data = data[:20000]

# plt.figure(figsize=(35,5))
# plt.plot(clean_data)
# plt.show()

In [5]:
scaler = MinMaxScaler(feature_range=(0, 5)) #init scaler, scales between 0 and 5
Y_old = scaler.fit_transform(np.asarray([i for i in range(len(clean_data)+1)]).reshape(-1,1)) #generate supervised output data

#plot the output with points for ints - represents the end of a cycle and an increment in count
# plt.figure(figsize=(35,5))
# plt.plot(Y_old)
# plt.scatter([4000,4000*2,4000*3,4000*4,4000*5],[1,2,3,4,5],color='Black')
# plt.show()

In [6]:
actions = np.asarray([filter_signal(clean_data)[:4001]]) #equivalent to actions
states = np.asarray(Y_old[:4001].copy()).T #equivalent to states

x_new = np.asarray([actions[0][:4001]])
y_new = np.asarray([states[0][:4001]])

print(x_new.shape)
print(y_new.shape)

(1, 4001)
(1, 4001)


In [7]:
#Linear Model
# def model(theta,a_t,prev_out):
#     return theta[0] + theta[1]*a_t + theta[2]*prev_out
def model2(theta,m1,mprev):
    return ((theta[7]-1)/(theta[7]))*mprev + (1/theta[7])*m1

#Exponential Model
def model(theta,a_t,prev_out):
#     return theta[0]*prev_out + (1-theta[0])*a_t + theta[1]
#     return theta[0]*prev_out + theta[1]*a_t 
#     return theta[0]*prev_out + math.log(prev_out,theta[1]) + theta[2]*a_t + math.log(prev_out,theta[3])
#     return theta[0]*prev_out + np.log(prev_out) / np.log(theta[1]) + theta[2]*a_t + np.log(a_t) / np.log(theta[3])
#     return theta[0]*prev_out + theta[2]*a_t + theta[3]
#     return theta[0]*prev_out**theta[1] + theta[2]*a_t 

    temp = theta[0] + theta[1]*np.sin(theta[2]*prev_out) + theta[3]*np.sin(theta[4]*a_t) + theta[5]*prev_out + theta[6]*a_t
    return model2(theta,temp,prev_out)

#     return theta[0]+theta[1]*prev_out+theta[2]*a_t+ prev_out**(.5+theta[3])+a_t**(.5+theta[4])


# loop for training model over all input/output action/state pairs
def loop(theta,s,a, prev_out):
    pr = prev_out
    # compute least squares over all imitator model outputs at once
    s_predict = [s[:,0]]  # container for system_model state outputs
    for t in range(a.shape[1]-1):

        # get current action-state pair
        s_t = s[:,t]
        a_t = a[:,t]

        # feed into system_model to get predicted output
        modelout = model(theta,a_t,pr)
        s_hat = modelout
        
        # store prediction
        s_predict.append(s_hat)
        pr = s_hat
        
    # array-afy predictions and return
    return np.array(s_predict).T

# an implementation of the least squares cost for system identification
# note here: s is an (1 x T) array and a an (1 x T-1) array
def least_squares(theta,s,a):
#     print("Least Squares Called.")
    # loop - runs over all action-state pairs and produces entire
    # state prediction set
    p = 0.1
    s_predict1 = loop(theta,s,a,p)
#     s_predict = loop2(theta,s,a)
    
    # compute least squares error between real and predicted states
    cost = np.sum((s_predict1[:,1:] - s[:,1:])**2)
    return cost/float(s.shape[1]-1)

# a simple initializer for this model
def initializer():
#     return 1e-3*np.random.randn(8,1)
    return np.ones((8,1))
#     return np.array([[5]])

#1+2*maxits calls to least square
def gradient_descent(g,alpha,max_its,w): 
    # compute gradient module using autograd
    gradient = grad(g)

    # run the gradient descent loop
    weight_history = [w] # weight history container
    cost_history = [g(w,actions,states)]
    
    for k in range(max_its):

#         if 5%5==0: print("Iteration",k)
        print("Iteration",k)
            
        # evaluate the gradient
        grad_eval = gradient(w,actions,states)

        # take gradient descent step
        w = w - alpha*grad_eval
        
        # record weight and cost
        weight_history.append(w)
        cost_history.append(g(w,actions,states))
        
    return weight_history,cost_history

In [ ]:
g=least_squares;
w=initializer();
k=3; alph=1e-3;
weight_history_1,cost_history_1 = gradient_descent(g,alph,k,w)

Iteration 0
Iteration 1


In [ ]:
fig = plt.figure(figsize=(10,5))
plt.plot(range(len(cost_history_1)),cost_history_1,c='red',label='alpha = '+str(alph))
plt.legend(loc='upper right')
plt.title("Cost Function History Plot")
plt.xlabel("Iteration Number")
plt.ylabel("Cost Function 'g(w)'")
plt.show()

In [ ]:
w1 = weight_history_1[-1]
p = 0.1
sp=[]
mywave=[]
for i in x_new:
    temp = model(w1,i,p)
    mywave.append(temp)
    #sp.append(mywave)
    p = temp

# mywave= [model(w1,x_new,p)]

fig = plt.figure(figsize=(10,5))

plt.plot(y_new[0],c='red')
plt.plot(mywave[0],c="green")

plt.title("Prediction vs actual data")
plt.show()

In [ ]:
weight_history_1